In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 2]
%matplotlib widget
import seaborn as sns

import sc3nb as scn
from pya import Asig

In [ ]:
import sonecules

In [ ]:
%run prepare-data.ipynb

### ICAD 2023
# sonecules: A Python Sonfication Architecture
### Dennis Reinsch and Thomas Hermann
#### Ambient Intelligence Group, Bielefeld University, Bielefeld, Germany

# Why sonecules?

<img src="figures/Sonecules-Motivation1.png" width="100%">

<img src="figures/Sonecules-Motivation2.png" width="100%">

# sonecules: Toolchain Solution

<center>
<img src="./figures/ecosystem-transpose.drawio.svg" width=100%/>
</center>

- inspired by chemistry: where molecules are combinations of atoms
- Likewise sonecules are combinations of single parts

## Sonecules: Getting Started

In [ ]:
import sonecules

Now we startup, which uses the default backend (sc3/ resp. sc3nb).

In [ ]:
sonecules.startup()

Next let's load some data (as pandas Dataframes) to demo our Sonecules

In [ ]:
%run prepare-data.ipynb

### Fundamental Concepts: Context and Timeline

In [ ]:
context = sonecules.gcc()

# create a synth
s1 = context.synths.create("s1", mutable=False)    

# schedule sonic marks
with context.at(0.4): s1.start(freq=400, dur=0.2)
with context.at(0.6): s1.start(freq=500, dur=1.0)
with context.at(0.8): s1.start(freq=600, dur=1.0)

context.timeline.plot()

### Fundamental Concepts: Playback

In [ ]:
context.timeline.plot()

In [ ]:
playback = sonecules.playback()  # used as media player

In [ ]:
playback.start()  

In [ ]:
playback.time

In [ ]:
playback.time = 0.3

In [ ]:
playback.stop()

# Sonecules Demonstration

* Now we use Sonecules, i.e. capsuled sonification classes
* which give a more high-level access to sonification methods
* Let's look at 
  * Audification
  * Data modulating sound parameters at high frequencies
  * Time-Variant Oscillators as Continuous Mappings 
  * Standard Discrete and Continuous Parameter Mapping Sonifications
  * Model-based Sonification: Data Sonogram

### Audification Sonecule

In [ ]:
from sonecules.buffersyn import Audification

Let's test with EEG data (of an epileptic seizure).
Here is the data

In [ ]:
dasig = Asig(eeg_data, sr=250)
plt.figure(); dasig.plot(offset=1, color='r', lw=0.5);

In [ ]:
audification = Audification(data=eeg_data[:,[0,8]], sr=256)

In [ ]:
context.clear()
audification.schedule(0, params={"rate": 7})
playback.start()

You can check more parameters of the used synth:

In [ ]:
audification.synth.params

In [ ]:
context.clear()
audification.schedule(0, params={"rate": 20, "amp": 0.5, "pan": 0.75})
playback.start()

## Score-based mappings

Let's select some data for the example

In [ ]:
# take segment at onset of epilepsy, only selected channels, decimate by 5
data = Asig(eeg_data, sr=250)[{7.5:10.5}, [0,1,2,5,9,12]][::5]
plt.figure(); data.plot(offset=2);

### Time Variant Oscillator

In [ ]:
from sonecules.scoresyn import TVOSon

snctvo = TVOSon(dasig[{7.5: 10.5}, [1, 2, 3]][::2])

context.clear()
snctvo.schedule(at=0, rate=4,
    base_pitch=60,
    pitch_step=12,
    pitch_relwid=3,
    amp_mode="change",
    level=-10,
    map_mode="channelwise",
)
playback.start(rate=1)

### Special Case: Polyphonic Sonification

In [ ]:
snctvo = TVOSon(dasig[{7.5: 11.5}, [0,1,2,3,4,5]][::2])
context.clear()
snctvo.schedule(at=0, rate=0.5,
    base_pitch=50,
    pitch_step=[0,4,7,12,16,19],  # selected musical tones for channels
    pitch_relwid=0.0,  # try 0.5 
    amp_mode="change",
    level=-5,
    map_mode="channelwise",
)
playback.start(rate=1)

### Special Case: Timbral Sonification (and more)

In [ ]:
# timbral sonification is just a special case of TVOSon
snctvo = TVOSon(dasig[{7.5: 9.5}, :][::2])

f0 = 60
base_pitch = scn.cpsmidi(f0)
pitch_steps = [ scn.cpsmidi(f0*(i+1)) - base_pitch for i in range(dasig.channels)] 

context.clear()
snctvo.schedule(at=0, rate=0.3,
    base_pitch=base_pitch,
    pitch_step=pitch_steps,
    pitch_relwid=0,  # use 1.5 for pitch added effect
    amp_mode="change",
    level=-10,
    map_mode="channelwise",
)
playback.start(rate=1)

In [ ]:
sonecules.stop()

This special case works via a score: 
- each data involves a set event on a synth (sent via OSC)
- but alternatively mapping could also be managed by buffers
- then each buffer directly modulates the amplitude of an oscillator
- For that we can already show a special Sonecule:

### Timbral-Sonification (as own sonecule via buffers)

In [ ]:
timbralson = sonecules.buffersyn.TimbralSon(eeg_data[14*256:16*256], sr=256)

context.clear()
timbralson.schedule(at=0.5, params={"amp": 0.05, "f0": 120, "rate": 0.5})
playback.start(rate=1)

- this scales better with high-frequency data (e.g. >1000 changes/second)
- a score-based approach would result in issues such as
  - Python CPU bottleneck
  - non-accurate (due to control rate being lower than audio rate)
- but this depends on the backend 

In [ ]:
sonecules.stop() 

## Classical Parameter Mapping Sonification

There are several ways of doing classical parameter mapping (both discrete and continuous)

In [ ]:
from sonecules.scoresyn import StandardContinuousPMSon, StandardDiscretePMSon
from sc3nb import midicps, linlin

Let's work with the penguins data set (measures of penguins)

In [ ]:
penguins_df.head(6)

In [ ]:
# penguins_df[['body_mass_g', 'flipper_length_mm']).plot()
sns.pairplot(data=penguins_df, hue="species", x_vars='body_mass_g', y_vars='flipper_length_mm');

In [ ]:
context.clear()

scpmson = StandardContinuousPMSon("s2", 
    {"freq": {"bounds": (midicps(30), midicps(90))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)

dlinlin = lambda value, dmin, dmax, y1, y2: linlin(value, x1=dmin, x2=dmax, y1=y1, y2=y2)

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(40), "y2": midicps(80)})
}

scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)

playback.start()

In [ ]:
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 

In [ ]:
context.clear() 

sdpmson = StandardDiscretePMSon("s1", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1},
     #"dur": {"default": 0.2}
    }
)

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "dur": ("body_mass_g", dlinlin, {"y1": 0.05, "y2": 0.1}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(50), "y2": midicps(70)})
}

sdpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
playback.start()

In [ ]:
context.timeline.plot()

### Continuous Parameter Mapping with specifying a callback function

In [ ]:
from sonecules.scoresyn import CPMSonCB

In [ ]:
#plt.figure(); Asig(bld_df.iloc[:24*7, 8:].values, sr=24).plot(offset=1); plt.grid(); plt.title("Building Dataset");

In [ ]:
scb = CPMSonCB(bld_df.iloc[:7*24, 6:])  # one week
# scb = CPMSonCB(bld_df.iloc[:4000, 7:])  # whole dataset

mapcol = scb.mapcol

def callback_fn(r, cmi, cma, pp):
    pp['freq']      = scn.midicps(mapcol(r, 'temperature', cmi, cma, 48, 72))
    pp['amp']       = scn.dbamp(mapcol(r, 'humidity', cmi, cma, -30, -10)) 
    pp['pan']       = mapcol(r, 'hc_wb_electrical', cmi, cma, -1, 1)
    pp['numharm']   = mapcol(r, 'solar_radiation', cmi, cma, 1, 6)
    pp['vibfreq']   = scn.linlin(r['hc_wb_hot_water'], -0.5, 0.5, 3, 8)
    pp['vibintrel'] = 0
    return pp

context.clear()
scb.schedule(at=0, duration=4, callback_fn=callback_fn)
playback.start()

In [ ]:
scb.create_callback_template(auto_assign=True);

### Model-Based Sonification

In [ ]:
from sonecules.triggersyn import DataSonogram

In [ ]:
sonecules.reset()
sonecules.gcc().enable_realtime()

In [ ]:
s1.start(freq=np.random.random()*500, dur=1)

In [ ]:
dsg = DataSonogram(penguins_df, x="flipper_length_mm", y="bill_length_mm", label="species")

# Goals

## availability and distribution

* sonifications as reusable units
* growing (crowd sourced) collection

## reproducibility

* benchmarking
* enchancing scientific standards and development

## sonification to the masses

* spreading sonifications as quickly usable tools
* reaching and enabling people without sonification expertise


# 

# mesonic concepts

<center>
<img src="./figures/mesonic-concepts.jpg"/>
</center>

# matplotlib

<center>
<img src="./figures/anatomy-of-figure.jpg"/>
</center>

# mesonic - audio objects

<center>
<img src="./figures/mesonic-synth.jpg"/>
</center>

* practical use: discrete vs continuous Parameter Mapping Sonification

* inspired by Enge et al. 2021 - **0D / 1D auditory mark**

# mesonic - audio objects

<center>
<img src="./figures/mesonic-buffer-record.jpg"/>
</center>

- the Context can be regarded as counterpart of the Figure from matplotlib and as the central interface
    - it controls the backend which is clearly separated and designed to be exchangeable

- The Backend provides different Managers to create the Audio Objects
- and is also responsible for creating the Audio Object EventHandlers that will create sound in the backend 


- The Audio Objects  are the available building blocks for the sonification which are inspired by common concepts from sound synthesis software
- The different objects offer actions like f.e. starting a Synth with a certain frequency
  These however do not directly generate sounds but rather create Events 


- The events are then passed to the Context which provides the sonification time for the event.
  The Event then will be inserted into the timeline as a bundles 

- The Timeline then forms a data structure that contains all the actions from the sonification

- To actually listen to the sonification the events in the timeline can be rendered offline or by the playback

- The playback offers an interactive control over the sonification and allows f.e. filtering by data source via the BundleProcessor before passing the events to the Audio Object EventHandler in the Backend 

